## CoderHouse - Data Engineering - Kevin Schiebelbein - Entregable 1

> Antes de ejecutar el codigo se necesita instalar las dependencias con ` pip3 install -r requeriments.txt ` en la carpeta raiz del Notebook

Se importan las librearias necesarias

In [34]:
from utils import getFakeData
from dotenv import dotenv_values
import redshift_connector
import json

Se define la configuracion para la consulta al Data Warehouse

In [35]:
config = dotenv_values(".env")
driver = "Amazon Redshift (x64)"
host = "coderhousede.c9baaa4ijdny.us-east-2.redshift.amazonaws.com"
db = "dev"
user = "kschiebelbein"
password = config["PASSDW"]

Se extraen los datos de la API publica y se organizan los datos para la insercion

In [36]:
api = "https://fakestoreapi.com/products"
result = getFakeData(api)
products = json.loads(result.text)
values = [tuple((p["id"], p["title"], p["price"], p["category"], p["image"], p["rating"]["rate"])) for p in products]

Se insertan los datos en el destino

In [37]:
# Connects to Redshift cluster using AWS credentials
with redshift_connector.connect(host=host,database=db,user=user,password=password) as conn:
    with conn.cursor() as cursor:
      conn.autocommit = True
      cursor: redshift_connector.Cursor = conn.cursor()
      tabla = f"""
        CREATE TABLE IF NOT EXISTS public.products (
        id INTEGER,
        title VARCHAR(128),
        price FLOAT8,
        category VARCHAR(256),
        image TEXT,
        rate FLOAT8
        ) DISTKEY(id) SORTKEY(rate);
      """
      cursor.execute(tabla)
      try:
        cursor.executemany("insert into products (id, title, price, category, image, rate) values (%s, %s, %s, %s, %s, %s)", values)
      except Exception as e:
        print(f"Error al guardar los datos: {e}")